In [ ]:
import numpy as np
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')


## 데이터 수집: kaggle 데이터 불러오기

In [ ]:
from google.colab import files 
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"hyunjilee97","key":"23213e51292dcf38510875086d00ad5d"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

Processing /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303/kaggle-1.5.12-cp37-none-any.whl
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

house-prices-advanced-regression-techniques.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip house-prices-advanced-regression-techniques.zip -d input

Archive:  house-prices-advanced-regression-techniques.zip
replace input/data_description.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace input/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace input/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace input/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
path = '/content/input/'
house_df_org = pd.read_csv(path+"train.csv")
house_df = house_df_org.copy()
test = pd.read_csv(path+ 'test.csv')
sample_submission = pd.read_csv(path+'sample_submission.csv')

shape, info, head

In [ ]:
house_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
# train
print(house_df.shape)
print(house_df.dtypes.value_counts())
NA_col = house_df.isna().sum()
print(NA_col[NA_col>0].sort_values(ascending= False))

print("-"*40)

#test
print(test.shape)
print(test.dtypes.value_counts)
NA_col_t = test.isna().sum()
print(NA_col_t[NA_col_t>0].sort_values(ascending= False))


(1460, 81)
object     43
int64      35
float64     3
dtype: int64
PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageType        81
GarageFinish      81
GarageQual        81
GarageCond        81
BsmtFinType2      38
BsmtExposure      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
MasVnrType         8
Electrical         1
dtype: int64
----------------------------------------
(1459, 80)
<bound method IndexOpsMixin.value_counts of Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 80, dtype: object>
PoolQC          1456
MiscFeature     1408
Alley           1352
Fence           1169
FireplaceQu      730
LotFrontage      22

## 결측치 처리

In [ ]:
# 결측치가 너무 많은 컬럼 제거
drop_col = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu']
for df in [house_df, test]:
  df.drop(drop_col, axis=1, inplace= True)

#숫자형 컬럼 --> 중간값 대체
for df in [house_df, test]:
  df.fillna(df.median(),inplace= True )

In [ ]:
# 문자형 컬럼만 남음 --> 이후 get_dummies실행
NA_col = house_df.isna().sum()
print(house_df.dtypes[NA_col[NA_col>0].index])

print("-"*40)

NA_col_t = test.isna().sum()
print(test.dtypes[NA_col_t[NA_col_t>0].index])


MasVnrType      object
BsmtQual        object
BsmtCond        object
BsmtExposure    object
BsmtFinType1    object
BsmtFinType2    object
Electrical      object
GarageType      object
GarageFinish    object
GarageQual      object
GarageCond      object
dtype: object
----------------------------------------
MSZoning        object
Utilities       object
Exterior1st     object
Exterior2nd     object
MasVnrType      object
BsmtQual        object
BsmtCond        object
BsmtExposure    object
BsmtFinType1    object
BsmtFinType2    object
KitchenQual     object
Functional      object
GarageType      object
GarageFinish    object
GarageQual      object
GarageCond      object
SaleType        object
dtype: object


## 이상치 탐색

In [ ]:
numeric_features = house_df.dtypes[house_df.dtypes != 'object'].index
object_features = house_df.dtypes[house_df.dtypes =='object'].index

In [ ]:
for i in range(len(numeric_features)):
  print("----------------------------",numeric_features[i])
  # print(np.percentile(house_df[numeric_features[i]],[25,75]))
  # print(house_df[numeric_features[i]].min(), house_df[numeric_features[i]].max())
  q1, q3 = np.percentile(house_df[numeric_features[i]], [25,75])
  outlier = house_df[(house_df[numeric_features[i]]< q1)|(house_df[numeric_features[i]]>q3)]
  if outlier.shape[0] < 20:
    house_df.drop(outlier.index, axis=0, inplace= True)
    print("drop")

---------------------------- Id
---------------------------- MSSubClass
---------------------------- LotFrontage
---------------------------- LotArea
---------------------------- OverallQual
---------------------------- OverallCond
---------------------------- YearBuilt
---------------------------- YearRemodAdd
---------------------------- MasVnrArea
---------------------------- BsmtFinSF1
---------------------------- BsmtFinSF2
---------------------------- BsmtUnfSF
---------------------------- TotalBsmtSF
---------------------------- 1stFlrSF
---------------------------- 2ndFlrSF
---------------------------- LowQualFinSF
---------------------------- GrLivArea
---------------------------- BsmtFullBath
drop
---------------------------- BsmtHalfBath
---------------------------- FullBath
---------------------------- HalfBath
drop
---------------------------- BedroomAbvGr
---------------------------- KitchenAbvGr
---------------------------- TotRmsAbvGrd
---------------------------- Firep

In [ ]:
# 왜도 
from scipy.stats import skew
skewed_feats = house_df[numeric_features].apply(lambda x: skew(x))
skewed_feats = skewed_feats[ skewed_feats> 0.75]
print(skewed_feats.index[:-2])
# 왜도 높은 변수 --> 로그 변환
house_df[skewed_feats.index] = np.log1p(house_df[skewed_feats.index])
test[skewed_feats.index[:-2]] = np.log1p(test[skewed_feats.index[:-2]])

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtHalfBath', 'KitchenAbvGr', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'],
      dtype='object')


In [ ]:
non_skewed_feats = np.setdiff1d(numeric_features,skewed_feats.index)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
house_df[non_skewed_feats]= scaler.fit_transform(house_df[non_skewed_feats])
test[non_skewed_feats]= scaler.transform(test[non_skewed_feats])

## One- hot Encoding

In [ ]:
train_test = pd.concat([house_df,test])
train_test = pd.get_dummies(train_test, prefix = object_features, columns = object_features)

## Model 적용 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
house_df = train_test.iloc[:house_df.shape[0],:]
test = train_test.iloc[house_df.shape[0]:,:].drop('SalePrice', axis=1)

# 독립 종속 분할
X = house_df.drop("SalePrice", axis=1)
y= house_df["SalePrice"]
# train-test 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.3, random_state= True)

In [ ]:
rf = RandomForestRegressor()
lr = LinearRegression()
xgb = XGBRegressor()

models = [rf,lr,xgb]

In [ ]:
for model in models:
  model.fit(X_train,y_train)
  pred= model.predict(X_valid)
  score = mean_squared_error(y_valid,pred)
  print(model, score)

# pred= model.predict(test)
# pred = np.expm1(pred)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False) 0.021613100920409325
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False) 7.3280972269559e+16
[09:33:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missi

In [ ]:
sample_submission.iloc[:,1]= pred
sample_submission.to_csv("answer.csv", index = False)